<a href="https://colab.research.google.com/github/mmfara/Adversarial-Debiasing-Enhanced/blob/main/AD%2B_Github.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
!pip install aif360
!pip install aif360[LawSchoolGPA]
!pip install aif360[Reductions]
from aif360.datasets import BinaryLabelDataset
from aif360.algorithms.preprocessing import DisparateImpactRemover
!pip install BlackBoxAuditing

pip install 'aif360[inFairness]'


In [ ]:
# Import necessary libraries
from aif360.datasets import StandardDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.algorithms.preprocessing.optim_preproc import OptimPreproc
from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import load_preproc_data_adult
from aif360.algorithms.preprocessing.optim_preproc_helpers.distortion_functions import get_distortion_adult
from aif360.algorithms.preprocessing.optim_preproc_helpers.opt_tools import OptTools
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from aif360.algorithms.inprocessing import ExponentiatedGradientReduction
from aif360.algorithms.postprocessing.reject_option_classification import RejectOptionClassification
from aif360.metrics import ClassificationMetric

#STARTING OFF

In [ ]:
data = pd.read_csv("/content/COMPAS_race_gender_testing.csv")

In [ ]:
data.head()

,c_charge_degree,race,age_cat,score_text,sex,priors_count,decile_score,two_year_recid,juv_fel_count,juv_misd_count,juv_other_count
0,1,1,0,0,1,0,3,1,0,0,0
1,1,1,1,0,1,4,4,1,0,0,1
2,1,0,0,1,1,14,6,1,0,0,0
3,0,0,0,0,0,0,1,0,0,0,0
4,1,0,0,0,1,0,4,0,0,0,0


In [ ]:
X = data.drop(['two_year_recid'], axis =1)
y = data[['two_year_recid']]

In [ ]:
# Train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, shuffle =True, test_size=0.3, random_state=42)

In [ ]:
print(X_train.shape)
print(X_test.shape)

(3694, 10)
(1584, 10)


In [ ]:
# Create training DataFrame
train_df = X_train.copy()
train_df['two_year_recid'] = y_train

test_df = X_test.copy()
test_df['two_year_recid'] = y_test

In [ ]:
# Group by race and gender, then label
group_label_counts = train_df.groupby(['race', 'sex', 'two_year_recid']).size().unstack(fill_value=0)

# Optional: Rename columns for clarity
group_label_counts.columns = ['Negative (0)', 'Positive (1)']

# Display
print(group_label_counts)


          Negative (0)  Positive (1)
race sex                            
0    0             212           114
     1             681           459
1    0             250           141
     1             813          1024


In [ ]:
# Check value counts by group and label
group_label_counts_test = test_df.groupby(['race', 'sex', 'two_year_recid']).size().unstack(fill_value=0)

# Optional: Rename columns for clarity
group_label_counts_test.columns = ['Negative (0)', 'Positive (1)']

# Display
print(group_label_counts_test)


          Negative (0)  Positive (1)
race sex                            
0    0             100            56
     1             288           193
1    0              96            62
     1             355           434


In [ ]:
import pandas as pd
from imblearn.over_sampling import RandomOverSampler, SMOTE

# Define target and protected attribute columns
target_col = 'two_year_recid'
protected_cols = ['race', 'sex']  # Intersectional attributes

# Add label back to training data temporarily
X_train_temp = X_train.copy()
X_train_temp[target_col] = y_train

# Create a composite group identifier
X_train_temp['group'] = X_train_temp[protected_cols].astype(str).agg('_'.join, axis=1)

# Pick a sampler
sampler = RandomOverSampler(random_state=42)
# sampler = SMOTE(random_state=42)

resampled_X = []
resampled_y = []

# Oversample within each intersectional group
for group in X_train_temp['group'].unique():
    group_data = X_train_temp[X_train_temp['group'] == group]

    Xg = group_data.drop(columns=[target_col])
    yg = group_data[target_col]

    # Drop group before resampling
    Xg_no_group = Xg.drop(columns=['group'], errors='ignore')

    if yg.nunique() < 2:
        print(f"Skipping group {group}: only one class present.")
        continue

    Xg_res, yg_res = sampler.fit_resample(Xg_no_group, yg)

    # Restore the group column after resampling
    Xg_res['group'] = group
    resampled_X.append(Xg_res)
    resampled_y.append(pd.Series(yg_res))

# Combine everything
X_train_bal = pd.concat(resampled_X).reset_index(drop=True)
y_train_bal = pd.concat(resampled_y).reset_index(drop=True)

# Reattach target label
X_train_bal[target_col] = y_train_bal

# View final balanced counts per intersectional group
print("Final balanced class counts per (race, sex):")
X_train_bal[['group', target_col]].groupby(['group', target_col]).size().unstack(fill_value=0)


✅ Final balanced class counts per (race, sex):


two_year_recid,0,1
group,,
0_0,212,212
0_1,681,681
1_0,250,250
1_1,1024,1024


In [ ]:
X_train_bal.drop(columns=['group'], inplace=True)

In [ ]:
dataset_train = X_train_bal.reset_index(drop=True)
dataset_test = test_df.reset_index(drop=True)

In [ ]:
dataset_train.shape

(4334, 11)

In [ ]:
dataset_test.shape

(1584, 11)

Checking my datasets

In [ ]:
dataset_train

,c_charge_degree,race,age_cat,score_text,sex,priors_count,decile_score,juv_fel_count,juv_misd_count,juv_other_count,two_year_recid
0,0,0,1,1,1,0,5,0,0,0,0
1,1,0,0,0,1,2,1,0,0,0,0
2,1,0,0,2,1,10,9,0,0,0,0
3,1,0,2,0,1,0,1,0,0,0,0
4,1,0,1,2,1,0,8,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
4329,1,1,0,2,0,3,9,0,0,0,1
4330,1,1,0,1,0,1,6,0,0,0,1
4331,1,1,0,2,0,14,10,0,0,0,1
4332,1,1,0,0,0,2,3,0,0,0,1


In [ ]:
# Define favorable and unfavorable labels
favorable_label = 1
unfavorable_label = 0

privileged_groups = [{'race': 1, 'sex': 1}]

unprivileged_groups = [
    {'race': r, 'sex': s}
    for r in data['race'].unique()
    for s in data['sex'].unique()
    if not (r == 1 and s == 1)]


#DisparateImpactRemover

In [ ]:
import numpy as np
import logging
from typing import List, Union, Set
from aif360.algorithms import Transformer


class DisparateImpactRemover(Transformer):
    """
    Enhanced Disparate Impact Remover with:
    - Intersectionality support
    - min_group_size filtering
    - groups_to_repair selective control
    - Global repair (single Repairer call)
    - Robust verbose logging
    """

    def __init__(self,
                 repair_level: float = 1.0,
                 sensitive_attribute: Union[str, List[str], None] = None,
                 min_group_size: int = 0,
                 groups_to_repair: Union[List[str], Set[str], None] = None,
                 verbose: bool = True):
        super().__init__()
        from BlackBoxAuditing.repairers.GeneralRepairer import Repairer
        self.Repairer = Repairer

        if not 0.0 <= repair_level <= 1.0:
            raise ValueError("'repair_level' must be between 0.0 and 1.0.")
        self.repair_level = repair_level
        self.min_group_size = max(0, min_group_size)
        self.groups_to_repair = set(groups_to_repair) if groups_to_repair else None
        self.verbose = verbose

        # Handle sensitive attributes
        if sensitive_attribute is None:
            self.sensitive_attributes = []
        elif isinstance(sensitive_attribute, str):
            self.sensitive_attributes = [sensitive_attribute]
        elif isinstance(sensitive_attribute, list):
            self.sensitive_attributes = sensitive_attribute
        else:
            raise TypeError("sensitive_attribute must be str, list, or None")

        # Robust logging setup
        self.logger = logging.getLogger("DisparateImpactRemover")
        if self.verbose:
            self.logger.setLevel(logging.INFO)
            if not self.logger.hasHandlers():
                handler = logging.StreamHandler()
                formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
                handler.setFormatter(formatter)
                self.logger.addHandler(handler)

    def fit_transform(self, dataset):
        features = dataset.features.copy()
        repaired = dataset.copy()

        if not self.sensitive_attributes:
            self.sensitive_attributes = dataset.protected_attribute_names[:1]

        protected_indices = [dataset.feature_names.index(attr) for attr in self.sensitive_attributes]

        # Generate readable group labels like "group=1" or "gender=0|race=1"
        combined_groups = self._get_group_labels(features, dataset)
        unique, counts = np.unique(combined_groups, return_counts=True)

        if self.verbose:
            self._log_group_stats(unique, counts)

        # Determine which groups to include
        valid_groups = []
        for grp, cnt in zip(unique, counts):
            reason = []
            if cnt < self.min_group_size:
                reason.append(f"size {cnt} < {self.min_group_size}")
            if self.groups_to_repair is not None and grp not in self.groups_to_repair:
                reason.append("not in groups_to_repair")

            if reason:
                self.logger.info(f"Skipped group: {grp} — {'; '.join(reason)}")
            else:
                valid_groups.append(grp)
                self.logger.info(f"Included group: {grp} (size={cnt})")

        # Select only rows from valid groups
        group_mask = np.isin(combined_groups, valid_groups)
        if not np.any(group_mask):
            raise ValueError("No valid rows to repair after filtering.")

        filtered_features = features[group_mask]
        filtered_labels = combined_groups[group_mask]

        # Map valid group labels to numeric codes
        label_to_code = {label: i for i, label in enumerate(sorted(set(filtered_labels)))}
        group_codes = np.array([label_to_code[label] for label in filtered_labels])

        # Append group code to filtered features
        repair_input = np.hstack([filtered_features, group_codes.reshape(-1, 1)])
        repair_index = repair_input.shape[1] - 1

        # Run Repairer
        repaired_filtered = np.array(
            self.Repairer(repair_input.tolist(), repair_index, self.repair_level, False).repair(repair_input.tolist()),
            dtype=np.float64
        )

        # FIXED: Apply repair only to valid rows (no shape mismatch!)
        repaired_features = features.copy()
        repaired_features[group_mask] = repaired_filtered[:, :-1]

        # Restore protected attributes
        for idx in protected_indices:
            repaired_features[:, idx] = dataset.features[:, idx]

        repaired.features = repaired_features
        return repaired

    def _get_group_labels(self, features: np.ndarray, dataset) -> np.ndarray:
        """Create human-readable group labels from protected attributes"""
        indices = [dataset.feature_names.index(attr) for attr in self.sensitive_attributes]
        protected_names = [dataset.feature_names[idx] for idx in indices]
        return np.array([
            '|'.join(f"{protected_names[j]}={features[i, idx]}"
                     for j, idx in enumerate(indices))
            for i in range(features.shape[0])
        ])

    def _log_group_stats(self, unique, counts):
        """Print stats for all groups"""
        self.logger.info("\n=== Group Analysis ===")
        self.logger.info(f"Protected attributes: {self.sensitive_attributes}")
        self.logger.info(f"Minimum group size: {self.min_group_size}")
        if self.groups_to_repair:
            self.logger.info(f"Specific groups to repair: {self.groups_to_repair}")
        self.logger.info(f"\n{'Status'.ljust(12)} {'Group'.ljust(40)} Size")
        for grp, cnt in sorted(zip(unique, counts), key=lambda x: -x[1]):
            status = []
            if cnt < self.min_group_size:
                status.append("TOO SMALL")
            if self.groups_to_repair and grp not in self.groups_to_repair:
                status.append("NOT SELECTED")
            status_str = "|".join(status) if status else "PROCESS"
            self.logger.info(f"{status_str.ljust(12)} {grp.ljust(40)} {cnt}")


In [ ]:
# Applying DisparateImpactRemover
dir = DisparateImpactRemover(
    repair_level=0.4,
    sensitive_attribute=['race', 'sex'],
    min_group_size=100,
    verbose=True
)

In [ ]:
dataset_train_bld = BinaryLabelDataset(df=dataset_train,
                             label_names=['two_year_recid'],
                             favorable_label=favorable_label,
                             unfavorable_label=unfavorable_label,
                             protected_attribute_names=['race', 'sex'],
                             privileged_protected_attributes=privileged_groups)


dataset_test_bld = BinaryLabelDataset(df=dataset_test,
                                      label_names=['two_year_recid'],
                                      favorable_label=favorable_label,
                                      unfavorable_label=unfavorable_label,
                                      protected_attribute_names=['race', 'sex'],
                                      privileged_protected_attributes=privileged_groups)

###Fitting disparate impact removal on the dataset

In [ ]:
# Fit and transform the training data
#from aif360.algorithms.preprocessing import DisparateImpactRemover
dir_dataset = dir.fit_transform(dataset_train_bld)

In [ ]:
#Disparate Impact Remover transforms dataset
dir_dataset

               instance weights        features                              \
                                                protected attribute           
                                c_charge_degree                race age_cat   
instance names                                                                
0                           1.0             0.0                 0.0     0.6   
1                           1.0             0.6                 0.0     0.0   
2                           1.0             0.6                 0.0     0.0   
3                           1.0             0.6                 0.0     1.2   
4                           1.0             0.6                 0.0     0.6   
...                         ...             ...                 ...     ...   
4329                        1.0             0.6                 1.0     0.0   
4330                        1.0             0.6                 1.0     0.0   
4331                        1.0             0.6     

#AL

In [ ]:
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

In [ ]:
import numpy as np
import pandas as pd

try:
    import tensorflow.compat.v1 as tf
    tf.disable_v2_behavior()
except ImportError as error:
    from logging import warning
    warning("{}: AdversarialDebiasing will be unavailable. To install, run:\n"
            "pip install 'aif360[AdversarialDebiasing]'".format(error))

from aif360.algorithms import Transformer

class AdversarialDebiasing(Transformer):
    def __init__(self,
                 unprivileged_groups,
                 privileged_groups,
                 scope_name,
                 sess,
                 seed=None,
                 adversary_loss_weight=0.1,
                 num_epochs=50,
                 batch_size=128,
                 classifier_num_hidden_units=200,
                 debias=True):

        super(AdversarialDebiasing, self).__init__(
            unprivileged_groups=unprivileged_groups,
            privileged_groups=privileged_groups)

        self.scope_name = scope_name
        self.seed = seed

        self.unprivileged_groups = unprivileged_groups
        self.privileged_groups = privileged_groups

        # Allow multiple unprivileged/privileged group values
        protected_attrs = set()
        for group in (self.unprivileged_groups + self.privileged_groups):
            protected_attrs.update(group.keys())

        if len(protected_attrs) != 1:
            raise ValueError("Only one protected attribute can be handled at a time.")

        self.protected_attribute_name = list(protected_attrs)[0]

        self.sess = sess
        self.adversary_loss_weight = adversary_loss_weight
        self.num_epochs = num_epochs
        self.batch_size = batch_size
        self.classifier_num_hidden_units = classifier_num_hidden_units
        self.debias = debias

        self.features_dim = None
        self.features_ph = None
        self.protected_attributes_ph = None
        self.true_labels_ph = None
        self.pred_labels = None

    def _classifier_model(self, features, features_dim, keep_prob):
        with tf.variable_scope("classifier_model"):
            W1 = tf.get_variable('W1', [features_dim, self.classifier_num_hidden_units],
                                 initializer=tf.initializers.glorot_uniform(seed=self.seed1))
            b1 = tf.Variable(tf.zeros(shape=[self.classifier_num_hidden_units]), name='b1')
            h1 = tf.nn.relu(tf.matmul(features, W1) + b1)
            h1 = tf.nn.dropout(h1, keep_prob=keep_prob, seed=self.seed2)

            W2 = tf.get_variable('W2', [self.classifier_num_hidden_units, 1],
                                 initializer=tf.initializers.glorot_uniform(seed=self.seed3))
            b2 = tf.Variable(tf.zeros(shape=[1]), name='b2')

            pred_logit = tf.matmul(h1, W2) + b2
            pred_label = tf.sigmoid(pred_logit)

        return pred_label, pred_logit

    def _adversary_model(self, pred_logits, true_labels, num_classes):
        with tf.variable_scope("adversary_model"):
            s = tf.sigmoid(pred_logits)
            concat_input = tf.concat([s, s * true_labels, s * (1.0 - true_labels)], axis=1)
            input_dim = concat_input.shape[1]

            W = tf.get_variable('W_adv', [input_dim, num_classes],
                                initializer=tf.initializers.glorot_uniform(seed=self.seed4))
            b = tf.Variable(tf.zeros(shape=[num_classes]), name='b_adv')

            logits = tf.matmul(concat_input, W) + b
            preds = tf.nn.softmax(logits)

        return preds, logits

    def fit(self, dataset):
        if tf.executing_eagerly():
            raise RuntimeError("AdversarialDebiasing does not work in eager execution mode.")

        if self.seed is not None:
            np.random.seed(self.seed)
        ii32 = np.iinfo(np.int32)
        self.seed1, self.seed2, self.seed3, self.seed4 = np.random.randint(ii32.min, ii32.max, size=4)

        temp_labels = dataset.labels.copy()
        if not np.array_equal(np.unique(temp_labels), [0.0, 1.0]):
            temp_labels[(dataset.labels == dataset.favorable_label).ravel(), 0] = 1.0
            temp_labels[(dataset.labels == dataset.unfavorable_label).ravel(), 0] = 0.0

        if np.isnan(dataset.features).any() or np.isinf(dataset.features).any():
            raise ValueError("Features contain NaN or Inf values. Please clean the data.")

        with tf.variable_scope(self.scope_name):
            num_train_samples, self.features_dim = np.shape(dataset.features)

            self.features_ph = tf.placeholder(tf.float32, shape=[None, self.features_dim])
            self.protected_attributes_ph = tf.placeholder(tf.int32, shape=[None])
            self.true_labels_ph = tf.placeholder(tf.float32, shape=[None, 1])
            self.keep_prob = tf.placeholder(tf.float32)

            self.pred_labels, pred_logits = self._classifier_model(self.features_ph, self.features_dim, self.keep_prob)
            pred_labels_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=self.true_labels_ph, logits=pred_logits))

            unique_prot_attr = np.unique(dataset.protected_attributes[:, dataset.protected_attribute_names.index(self.protected_attribute_name)])
            num_classes = len(unique_prot_attr)

            if self.debias:
                pred_protected_attributes_labels, pred_protected_attributes_logits = self._adversary_model(
                    pred_logits, self.true_labels_ph, num_classes)
                pred_protected_attributes_loss = tf.reduce_mean(
                    tf.nn.sparse_softmax_cross_entropy_with_logits(
                        labels=self.protected_attributes_ph,
                        logits=pred_protected_attributes_logits))

            global_step = tf.Variable(0, trainable=False)
            starter_learning_rate = 0.0001  # More stable
            learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, 1000, 0.96, staircase=True)
            classifier_opt = tf.train.AdamOptimizer(learning_rate)
            if self.debias:
                adversary_opt = tf.train.AdamOptimizer(learning_rate)

            classifier_vars = [var for var in tf.trainable_variables(scope=self.scope_name) if 'classifier_model' in var.name]
            if self.debias:
                adversary_vars = [var for var in tf.trainable_variables(scope=self.scope_name) if 'adversary_model' in var.name]
                adversary_grads = {var: grad for (grad, var) in adversary_opt.compute_gradients(pred_protected_attributes_loss, var_list=classifier_vars)}

            normalize = lambda x: x / (tf.norm(x) + np.finfo(np.float32).tiny)

            classifier_grads = []
            for (grad, var) in classifier_opt.compute_gradients(pred_labels_loss, var_list=classifier_vars):
                if self.debias:
                    unit_adv_grad = normalize(adversary_grads[var])
                    grad -= tf.reduce_sum(grad * unit_adv_grad) * unit_adv_grad
                    grad -= self.adversary_loss_weight * adversary_grads[var]
                grad = tf.clip_by_value(grad, -5.0, 5.0)  # Gradient clipping
                classifier_grads.append((grad, var))

            classifier_minimizer = classifier_opt.apply_gradients(classifier_grads, global_step=global_step)
            if self.debias:
                with tf.control_dependencies([classifier_minimizer]):
                    adversary_minimizer = adversary_opt.minimize(pred_protected_attributes_loss, var_list=adversary_vars)

            self.sess.run(tf.global_variables_initializer())
            self.sess.run(tf.local_variables_initializer())

            for epoch in range(self.num_epochs):
                shuffled_ids = np.random.choice(num_train_samples, num_train_samples, replace=False)
                for i in range(num_train_samples // self.batch_size):
                    batch_ids = shuffled_ids[self.batch_size * i: self.batch_size * (i + 1)]
                    batch_features = dataset.features[batch_ids]
                    batch_labels = np.reshape(temp_labels[batch_ids], [-1, 1])
                    batch_protected_attributes = dataset.protected_attributes[batch_ids][:,
                        dataset.protected_attribute_names.index(self.protected_attribute_name)]

                    feed_dict = {
                        self.features_ph: batch_features,
                        self.true_labels_ph: batch_labels,
                        self.protected_attributes_ph: batch_protected_attributes,
                        self.keep_prob: 0.8
                    }

                    if self.debias:
                        _, _, loss_cls, loss_adv = self.sess.run([
                            classifier_minimizer, adversary_minimizer,
                            pred_labels_loss, pred_protected_attributes_loss
                        ], feed_dict=feed_dict)
                        if i % 200 == 0:
                            print(f"epoch {epoch}; iter: {i}; classifier loss: {loss_cls}; adversarial loss: {loss_adv}")
                    else:
                        _, loss_cls = self.sess.run([
                            classifier_minimizer,
                            pred_labels_loss
                        ], feed_dict=feed_dict)
                        if i % 200 == 0:
                            print(f"epoch {epoch}; iter: {i}; classifier loss: {loss_cls}")
        return self

    def predict(self, dataset):
        if self.seed is not None:
            np.random.seed(self.seed)

        num_test_samples, _ = np.shape(dataset.features)
        samples_covered = 0
        pred_labels = []

        while samples_covered < num_test_samples:
            start = samples_covered
            end = min(start + self.batch_size, num_test_samples)
            batch_ids = np.arange(start, end)
            batch_features = dataset.features[batch_ids]
            batch_labels = np.reshape(dataset.labels[batch_ids], [-1, 1])

            feed_dict = {
                self.features_ph: batch_features,
                self.true_labels_ph: batch_labels,
                self.keep_prob: 1.0
            }

            pred_labels += self.sess.run(self.pred_labels, feed_dict=feed_dict)[:, 0].tolist()
            samples_covered += len(batch_features)

        dataset_new = dataset.copy(deepcopy=True)
        dataset_new.scores = np.array(pred_labels, dtype=np.float64).reshape(-1, 1)
        dataset_new.labels = (np.array(pred_labels) > 0.5).astype(np.float64).reshape(-1, 1)

        temp_labels = dataset_new.labels.copy()
        temp_labels[(dataset_new.labels == 1.0).ravel(), 0] = dataset.favorable_label
        temp_labels[(dataset_new.labels == 0.0).ravel(), 0] = dataset.unfavorable_label

        dataset_new.labels = temp_labels.copy()
        return dataset_new


Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

sess = tf.Session()

# Initialize adversarial debiasing
adv = AdversarialDebiasing(
    privileged_groups=[{'race': 1, 'sex': 1}],
    unprivileged_groups=[
        {'race': r, 'sex': s}
        for r in data['race'].unique()
        for s in data['sex'].unique()
        if not (r == 1 and s == 1)
    ],
    scope_name='adv_debias_race_sex8',
    sess=tf.Session(),
    debias=True,
    seed=101,
    num_epochs=50,
    batch_size=128,
    classifier_num_hidden_units=100,
)

# Fit on a BinaryLabelDataset that includes 'race' and 'sex' as protected attributes
adv.fit(dir_dataset)


In [ ]:
# Predict with the debias_model
predictions = adv.predict(dataset_test_bld)

## Intersectional Fairness Result

In [ ]:
import pandas as pd
import numpy as np
from itertools import product
from aif360.metrics import ClassificationMetric

# Assumes you defined this earlier:
# privileged_groups = [{'race': 1, 'sex': 1}]

# Get indices of protected attributes
race_idx = dataset_test_bld.feature_names.index('race')
sex_idx = dataset_test_bld.feature_names.index('sex')

# All intersectional (race, sex) combinations
race_vals = np.unique(dataset_test_bld.features[:, race_idx])
sex_vals = np.unique(dataset_test_bld.features[:, sex_idx])
group_combinations = list(product(race_vals, sex_vals))

# Collect results
results = []

for (race_val, sex_val) in group_combinations:
    current_group = {'race': race_val, 'sex': sex_val}

    if current_group in privileged_groups:
        continue  # Skip privileged group

    metric = ClassificationMetric(
        dataset_test_bld,
        predictions,
        unprivileged_groups=[current_group],
        privileged_groups=privileged_groups
    )

    # Manual subgroup accuracy
    mask = (
        (dataset_test_bld.features[:, race_idx] == race_val) &
        (dataset_test_bld.features[:, sex_idx] == sex_val)
    )
    y_true = dataset_test_bld.labels[mask]
    y_pred = predictions.labels[mask]
    accuracy = (y_true == y_pred).mean()

    results.append({
        'race': int(race_val),
        'sex': int(sex_val),
        'Statistical Parity Difference': metric.statistical_parity_difference(),
        'Equal Opportunity Difference': metric.equal_opportunity_difference(),
        'Disparate Impact': metric.disparate_impact(),
        'Predictive Equality Difference': metric.false_positive_rate_difference(),
        'Subgroup Accuracy': round(accuracy, 6)
    })

# Results table
df_results = pd.DataFrame(results)
display(df_results)


,race,sex,Statistical Parity Difference,Equal Opportunity Difference,Disparate Impact,Predictive Equality Difference,Subgroup Accuracy
0,0,0,-0.168007,-0.088134,0.746058,-0.107606,0.673077
1,0,1,-0.162636,-0.157064,0.754176,-0.078717,0.632017
2,1,0,-0.091977,-0.094470,0.860978,0.001144,0.607595


## Aggregate Fairness Result

In [ ]:
# Evaluate the classifier's performance with fairness intervention
metric_with_fairness = ClassificationMetric(
    dataset_test_bld,
    predictions,
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups
)

In [ ]:
print("Performance with fairness intervention (Reject Option Classification):")
print("Accuracy: {:.6f}".format(metric_with_fairness.accuracy()))
print("Disparate Impact: {:.6f}".format(metric_with_fairness.disparate_impact()))
print("Mean Difference: {:.6f}".format(metric_with_fairness.mean_difference()))

Performance with fairness intervention (Reject Option Classification):
Accuracy: 0.662879
Disparate Impact: 0.773809
Mean Difference: -0.149647


In [ ]:
# Calculate fairness metrics
statistical_parity_difference = metric_with_fairness.statistical_parity_difference()
disparate_impact = metric_with_fairness.disparate_impact()
equal_opportunity_difference = metric_with_fairness.equal_opportunity_difference()

In [ ]:
# Print out the metrics
print(f"Statistical Parity Difference: {statistical_parity_difference}")
print(f"Disparate Impact: {disparate_impact}")
print(f"Equal Opportunity Difference: {equal_opportunity_difference}")

Statistical Parity Difference: -0.14964727264031374
Disparate Impact: 0.7738090074459626
Equal Opportunity Difference: -0.1321736038051773


In [ ]:
# Print out the equality of odds metrics
print("Difference in True Positive Rates (Unprivileged - Privileged) = %f" % metric_with_fairness.true_positive_rate_difference())
print("Difference in False Positive Rates (Unprivileged - Privileged) = %f" % metric_with_fairness.false_positive_rate_difference())

Difference in True Positive Rates (Unprivileged - Privileged) = -0.132174
Difference in False Positive Rates (Unprivileged - Privileged) = -0.068845
